In [1]:
import geopandas as gpd

In [2]:
county_fips = '26127'
nsi_api_call = 'https://nsi.sec.usace.army.mil/nsiapi/structures?fips=' + county_fips
nsi_gdf = gpd.read_file(nsi_api_call)

In [3]:
nsi_gdf.head()

,fd_id,bid,occtype,st_damcat,bldgtype,found_type,cbfips,pop2amu65,pop2amo65,pop2pmu65,...,source,med_yr_blt,firmzone,o65disable,u65disable,x,y,ground_elv,ground_elv_m,geometry
0,532313663,86MMJR3R+2XQ-0-0-0-0,RES1-1SWB,RES,W,B,261270106004041,1,0,1,...,P,1979,NaN,0.22,0.07,-86.157500,43.602577,734.366493,223.834900,POINT (-86.15750 43.60258)
1,532266788,86MMHH4C+QR3-3-4-2-5,RES1-1SWB,RES,W,B,261270109002084,1,0,1,...,P,1969,NaN,0.22,0.07,-86.427965,43.556892,730.388394,222.622375,POINT (-86.42797 43.55689)
2,532266802,86MMHH47+563-4-3-2-2,RES1-1SNB,RES,W,S,261270109002084,1,0,1,...,P,1969,NaN,0.22,0.07,-86.436954,43.555399,693.972202,211.522720,POINT (-86.43695 43.55540)
3,532266804,86MMHH37+R77-1-3-1-4,RES1-1SNB,RES,M,C,261270109002084,0,0,0,...,P,1969,NaN,0.22,0.07,-86.436842,43.554536,687.386092,209.515274,POINT (-86.43684 43.55454)
4,532266806,86MMHH37+W5F-4-3-4-4,RES1-2SNB,RES,M,S,261270109002084,0,0,0,...,P,1969,NaN,0.22,0.07,-86.437092,43.554821,691.162342,210.666275,POINT (-86.43709 43.55482)


In [4]:
# check unique ids
nsi_gdf[['fd_id','bid']].astype(str).describe()

,fd_id,bid
count,17553,17553
unique,17553,17169
top,532313663,86MMGJQP+C6F-0-0-0-0
freq,1,23


In [5]:
# print columns
nsi_gdf.columns

Index(['fd_id', 'bid', 'occtype', 'st_damcat', 'bldgtype', 'found_type',
       'cbfips', 'pop2amu65', 'pop2amo65', 'pop2pmu65', 'pop2pmo65', 'sqft',
       'num_story', 'ftprntid', 'ftprntsrc', 'students', 'found_ht',
       'val_struct', 'val_cont', 'val_vehic', 'source', 'med_yr_blt',
       'firmzone', 'o65disable', 'u65disable', 'x', 'y', 'ground_elv',
       'ground_elv_m', 'geometry'],
      dtype='object')

In [6]:
# describe sqft
nsi_gdf['sqft'].describe()

count     17553.000000
mean       2943.144987
std        8611.828746
min          43.698700
25%        1195.400000
50%        1855.664000
75%        2873.900000
max      510810.000000
Name: sqft, dtype: float64

In [7]:
# how many buildings have missing sqft?
nsi_gdf['sqft'].isna().sum()

0

In [8]:
# group by occtype and count
nsi_gdf[['fd_id','occtype']].groupby('occtype').count()

,fd_id
occtype,
AGR1,70
COM1,139
COM10,1
COM2,90
COM3,120
COM4,742
COM5,26
COM6,4
COM7,21


In [9]:
# prepare file for HUA
# add column for guid based on fd_id
nsi_gdf['guid'] = nsi_gdf['fd_id'].astype(str)+'-'+nsi_gdf['bid'].astype(str)
# add archetype based on occtype
nsi_gdf['archetype'] = nsi_gdf['occtype'].astype(str)
# add source column
nsi_gdf['bldgsource'] = 'NSI2023-05-09'
# Make new dataframe with key columns
nsi_gdf_hua = nsi_gdf[['guid','archetype','bldgsource','sqft','geometry']]
nsi_gdf_hua.head()

,guid,archetype,bldgsource,sqft,geometry
0,532313663-86MMJR3R+2XQ-0-0-0-0,RES1-1SWB,NSI2023-05-09,1216.000,POINT (-86.15750 43.60258)
1,532266788-86MMHH4C+QR3-3-4-2-5,RES1-1SWB,NSI2023-05-09,2721.630,POINT (-86.42797 43.55689)
2,532266802-86MMHH47+563-4-3-2-2,RES1-1SNB,NSI2023-05-09,1551.440,POINT (-86.43695 43.55540)
3,532266804-86MMHH37+R77-1-3-1-4,RES1-1SNB,NSI2023-05-09,651.826,POINT (-86.43684 43.55454)
4,532266806-86MMHH37+W5F-4-3-4-4,RES1-2SNB,NSI2023-05-09,7516.920,POINT (-86.43709 43.55482)


In [10]:
# Check unique guid
nsi_gdf_hua['guid'].describe()

count                              17553
unique                             17553
top       532313663-86MMJR3R+2XQ-0-0-0-0
freq                                   1
Name: guid, dtype: object

In [11]:
# Save file in my WorkNPR folder
folderpath = "../../../WorkNPR/SourceData/"
filename = f"nsi_01av1_hua_{county_fips}.shp"
nsi_gdf_hua.to_file(folderpath + filename)